In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import dlc_practical_prologue as prologue

In [16]:
#Number of pairs
N = 2000
#Loading data
pre_train_input, train_target, train_classes, pre_test_input, test_target, test_classes = \
prologue.generate_pair_sets(N)

/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [14]:
"""train_input = torch.rand(N,1,28,14)
test_input = torch.rand(N,1,28,14)
for i in range(N):
    temp_train = torch.cat((pre_train_input[i,0,:,:], pre_train_input[i,1,:,:]),0)
    temp_test = torch.cat((pre_test_input[i,0,:,:], pre_test_input[i,1,:,:]),0)
    train_input[i] = temp_train
    test_input[i] = temp_test
train_input.size()"""

torch.Size([2000, 1, 28, 14])

In [8]:
class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 20, kernel_size=4)
        self.conv2 = nn.Conv2d(20, 800, kernel_size=5)
        self.fc1 = nn.Linear(2*2*800,150)
        self.fc2 = nn.Linear(150, 10)
        
        self.binary = nn.Linear(20, 2) #take as input the 2 digit prediction and return a binary classification
        
    def forward(self, x):
        # or use flatten
        res = []
        for i in range(2): #Siamese networks, shared weights
            _x = x[:, i, :, :]
            _x = F.relu(F.max_pool2d(self.conv1(_x), kernel_size = 3, stride = 3))
            _x = F.relu(F.max_pool2d(self.conv2(_x), kernel_size = 2, stride = 2))
            _x = self.fc1(_x.view(-1, 2*2*800))
            _x = F.softmax(_x)
            _x = self.fc2(_x)
            res.append(F.softmax(_x))
        res.view(-1,20)
        x = self.binary(res)
        
        return x


In [ ]:
def train(model, train_input, train_target, epochs, mini_batch_size):
    
    criterion = nn.BCELoss()
    eta = 1e-1
    
    model.train()
    
    for e in range(epochs):
        sum_loss = 0
        for b in range (0, train_input.size(0),mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()
            for p in model.parameters():
                p.data.sub_(eta * p.grad.data)
        print(e, sum_loss)
        
def compute_nb_errors(model, input, target, mini_batch_size):
    